<a href="https://colab.research.google.com/github/prasanth-ntu/pookie-llm-finetuning-resources/blob/main/finetuning/unsloth/ascii_art_completion_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Completion finetuning using unsloth

**Google Colab T4 resources**
- System RAM: 12.7 GB
- GPU RAM: 15.0 GB
- Disk: 112.6 GB

In [2]:
# !top

=top - 07:39:30 up 30 min,  0 users,  load average: 2.36, 1.04, 1.10
Tasks:  18 total,   1 running,  16 sleeping,   0 stopped,   1 zombie
%Cpu(s): 56.7 us, 10.0 sy,  0.0 ni, 30.0 id,  3.3 wa,  0.0 hi,  0.0 si,  0.0 st
MiB Mem :  12978.0 total,   5124.4 free,   1483.4 used,   6370.1 buff/cache
MiB Swap:      0.0 total,      0.0 free,      0.0 used.  11163.9 avail Mem 

    PID USER      PR  NI    VIRT    RES    SHR S  %CPU  %MEM     TIME+ COMMAND  
   9784 root      20   0 2264844 771704  41488 S 126.7   5.8   0:22.24 node     
      1 root      20   0    1076      8      0 S   0.0   0.0   0:00.04 docker-+ 
      7 root      20   0 1308384  79668  41020 S   0.0   0.6   0:06.21 node     
     15 root      20   0    7376   3416   3108 S   0.0   0.0   0:00.87 oom_mon+ 
     17 root      20   0    7376   1896   1612 S   0.0   0.0   0:00.00 run.sh   
     18 root      20   0 1238292  17376   9684 S   0.0   0.1   0:01.31 kernel_+ 
     37 root      20   0    5808   1096   1000 S   0.0   0.0 

In [1]:
# After loading the model
!nvidia-smi

Sat Jul 19 07:39:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

This notebook makes use of unsloth to finetune a model for a completion task.
In this example we will finetune the llama 3.2 base model to generate ascii art. I would recommend using the unsloth library compared to just using the huggingface library as it requires less memory and is faster.

Adapted from unsloth notebooks, if something is broken check on:
https://unsloth.ai/

In [1]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3  peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [ ]:
!pip install -U datesets

### Load base model

In [16]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/Llama-3.2-3B",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = False,
    token=userdata.get('HF_ACCESS_TOKEN')
)

==((====))==  Unsloth 2025.7.5: Fast Llama patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
tokenizer.clean_up_tokenization_spaces = False

### Add lora to base model and patch with Unsloth

In [18]:
# More info about parameters: https://huggingface.co/docs/peft/v0.11.0/en/package_reference/lora#peft.LoraConfig
target_modules =  ["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"]

# When adding special tokens
train_embeddings = False

if train_embeddings:
  target_modules = target_modules + ["lm_head"]

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # rank of lora matrices according to paper not much loss when set relatively low
    target_modules = target_modules,  # On which modules of the llm the lora weights are used
    lora_alpha = 16, # scales the weights of the adapters (more influence on base model), 16 was recommended on reddit
    lora_dropout = 0, # Default on 0.05 in tutorial but unsloth says 0 is better
    bias = "none",    # "none" is optimized
    use_gradient_checkpointing = "unsloth", #"unsloth" for very long context, decreases vram
    random_state = 3407,
    use_rslora = False,  # scales lora_alpha with 1/sqrt(r), huggingface says this works better
    loftq_config = None, # And LoftQ
)

Unsloth 2025.7.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [19]:
empty_prompt = """
{ascii_art}
"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func_no_prompt(examples):
  ascii_art_samples = examples["ascii"]
  training_prompts = []
  for ascii_art in ascii_art_samples:
      training_prompt = empty_prompt.format(ascii_art=ascii_art) + EOS_TOKEN
      training_prompts.append(training_prompt)
  return { "text" : training_prompts, }


from datasets import load_dataset
dataset = load_dataset("pookie3000/ascii-cats", split = "train", download_mode="force_redownload")
dataset = dataset.map(formatting_prompts_func_no_prompt, batched = True)

train-00000-of-00001.parquet:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/201 [00:00<?, ? examples/s]

Map:   0%|          | 0/201 [00:00<?, ? examples/s]

 ### Visualize dataset

In [20]:
dataset

Dataset({
    features: ['ascii', 'creature', 'text'],
    num_rows: 201
})

In [23]:
import random
random_index = random.randint(0, len(dataset) - 1)
print(f"Random index: {random_index}")
print(f"Content at random index:\n{dataset[random_index]}")

Random index: 14
Content at random index:
{'ascii': "      |\\      _,,,---,,_\nZZZzz /,`.-'`'    -.  ;-;;,_\n     |,4-  ) )-,_. ,\\ (  `'-'\n    '---''(_/--'  `-'\\_) ", 'creature': 'cat', 'text': "\n      |\\      _,,,---,,_\nZZZzz /,`.-'`'    -.  ;-;;,_\n     |,4-  ) )-,_. ,\\ (  `'-'\n    '---''(_/--'  `-'\\_) \n<|end_of_text|>"}


In [21]:
i = 5
print(dataset[i]["creature"])
print(dataset[i]["ascii"])


cat
 _                        
/ |                       
| |                       
| |                       
| |                   /|  
\ \                /~ ,\ 
\ \-------.....-'       X
|                 ~  .-~ 
|    \             /  
\   /___     _\  /   
| /   ~~~~~ /\ |   
| ||        / | |   
( ||       / /| |   
\_))      \_) \_)   



In [6]:
for i, sample in enumerate(dataset):
    print(f"\n------ Sample {i + 1} ----")
    print(sample["text"])
    if i > 2:
      break


------ Sample 1 ----


KeyError: 'text'

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # process 4 batches before updating parameters (parameter update == step)
        num_train_epochs = 5, # between 1 - 3 to prevent overfitting
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none"
    ),
)

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 201 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 125
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,0.708000
2,0.796500
3,1.094700
4,1.069200
5,1.514900
6,1.391900
7,1.822400
8,1.760000
9,1.230700
10,1.672400


### inference

In [ ]:
from transformers import TextStreamer

def generate_ascii_art(model):
    FastLanguageModel.for_inference(model)
    inputs = tokenizer("", return_tensors = "pt").to("cuda")
    text_streamer = TextStreamer(tokenizer)
    # https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/text_generation#transformers.GenerationMixin
    # https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/text_generation#transformers.GenerationConfig
    for token in model.generate(**inputs, streamer = text_streamer, max_new_tokens = 100):
        print(token)
        pass

In [ ]:
for _ in range(3):
  generate_ascii_art(model)

<|begin_of_text|>
 /\_/\
( o.o )
 > ^ <
<|end_of_text|>
tensor([128000,    198,  24445,  51395,   5779,      7,    297,  14778,   1763,
           871,   6440,  78665, 128001], device='cuda:0')
<|begin_of_text|>
  /\ ___ /\
 (  -   -  )
  \  ~ ~  /
  /       \
 /         \
 |         |
  \       /
  /// /// --
<|end_of_text|>
tensor([128000,    198,    220,  24445,   7588,    611,   5779,    320,    220,
           482,    256,    482,    220,   1763,    220,   1144,    220,   4056,
          4056,    220,  40081,    220,    611,    996,   3120,    611,    260,
          3120,    765,    260,   9432,    220,   1144,    996,  40081,    220,
          1066,   1066,  40614, 128001], device='cuda:0')
<|begin_of_text|>
  |\__/,|
  |o o  |
  ( T   )
 .^`^--'^.
 `.  ;  .'  
 | | | | |
((_((|))_))
<|end_of_text|>
tensor([128000,    198,    220,  64696,    565,  35645,   7511,    220,    765,
            78,    297,    220,   9432,    220,    320,    350,    256,   1763,
           662,     61,

## Saving

### Save lora adapter

This is both useful for inference and if you want to load the model again

In [ ]:
model.push_to_hub(
    "pookie3000/Llama-3.2-3B-ascii-cats-lora",
    tokenizer,
    token = userdata.get('HF_ACCESS_TOKEN')
)

### Merge model with lora weights and save to gguf

You can then do inference locally with Ollama or llama.cpp

##### Popular quantization methods

- **q4_k_m**  
  4bit quantization. Low memory. All models you pull with ollama uses this quantization.
- **q8_0**  
  8bit quantization. Medium memory.
- **f16**  
  16 bit quantization. A lot of models are already in 16 bit so then no quantization happens
- **not_quantized**  
  Often same as f16.

In [ ]:
model.push_to_hub_gguf(
    "pookie3000/Llama-3.2-3B-ascii-cats-lora-q4_k_m-GGUF",
    tokenizer,
    quantization_method="q4_k_m",
    token = userdata.get('HF_ACCESS_TOKEN')
)

README.md:   0%|          | 0.00/563 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

Saved model to https://huggingface.co/pookie3000/Llama-3.2-3B-ascii-cats-10-ep-lora


### Load model and saved lora adapters
For if you want to continue finetuning or want to do inference using the model in safetensor format.

In [ ]:
from transformers import TextStreamer

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="pookie3000/Llama-3.2-3B-ascii-cats-lora",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = False,
    token=userdata.get('HF_ACCESS_TOKEN')
)


def generate_ascii_art(model):
    FastLanguageModel.for_inference(model)
    inputs = tokenizer("", return_tensors = "pt").to("cuda")
    text_streamer = TextStreamer(tokenizer)
    # https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/text_generation#transformers.GenerationMixin
    # https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/text_generation#transformers.GenerationConfig
    for token in model.generate(**inputs, streamer = text_streamer, max_new_tokens = 100):
        print(token)
        pass
